In [1]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")

import pandas as pd
from src.data.pipelines import primary_use_pipeline, square_feet_pipeline

# Data preprocessing

In [2]:
building_metadata = pd.read_csv('data/building_metadata.csv')
building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


## Buildings data

### Primary use

In [3]:
primary_use_feature = pd.DataFrame(primary_use_pipeline.fit_transform(building_metadata))
primary_use_feature.head()

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


### Square feet

In [4]:
square_feet_feature = pd.DataFrame(square_feet_pipeline.fit_transform(building_metadata))
square_feet_feature.head()

,0
0,-0.764729
1,-0.807282
2,-0.783297
3,-0.617951
4,0.221212


## Weather data

In [5]:
train_weather = pd.read_csv('data/weather_train.csv')
train_weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
